In [1]:
import json
import math

# Load the GeoJSON data
with open('../kml/merged_test_1.geojson', 'r') as f:
    data = json.load(f)

# Loop over each feature in the GeoJSON data
for feature in data['features']:
    # Extract the coordinates of the fair territory polygon
    coordinates = feature['geometry']['coordinates'][0]

    # Find the index of the first coordinate, which should be home plate
    homeplate_index = -1
    min_distance = float('inf')
    for i, coord in enumerate(coordinates):
        distance = math.sqrt(coord[0]**2 + coord[1]**2)
        if distance < min_distance:
            min_distance = distance
            homeplate_index = i

    # Ensure that home plate is the first coordinate
    if homeplate_index != 0:
        coordinates = coordinates[homeplate_index:] + coordinates[:homeplate_index]

    # Calculate the distance from home plate to each point on the polygon
    distances = []
    for coord in coordinates:
        distance = math.sqrt(coord[0]**2 + coord[1]**2)
        distances.append(distance)

    # Print the distances
    print(distances)

# Convert the distances into feet
feet_per_meter = 3.28084


[94.21396007325428, 94.21396542356302, 94.21431753086247, 94.21446310711414, 94.21411436871225, 94.21411436871225, 94.21514598392382, 94.21511960463845, 94.21508419735912, 94.2144737439794, 94.21409044046283, 94.2140328587013]
[94.21411436871225, 94.21446310711414, 94.2145513800919, 94.21466624965085, 94.21475681093304, 94.2148056005321, 94.2148651476137, 94.21494489813891, 94.215032582736, 94.21511440371204, 94.2151741154855, 94.21521645300287, 94.21523682528914, 94.21524085829256, 94.21524767679098, 94.21524986517194, 94.21525275639385, 94.21525830648068, 94.21525118219716, 94.21524115602223, 94.21521133123595, 94.21517861992604, 94.21515696120653, 94.21514598392382, 94.21411436871225]
[93.93272001794192, 93.93272644921109, 93.93339582584477, 93.93339582584477, 93.93435687374034, 93.9343581380168, 93.93402482670454, 93.93371992444908, 93.93343465881148, 93.93331149968597, 93.93311650823003, 93.93292047258664]
[93.93272644921109, 93.93287989475378, 93.93294017761438, 93.93298456478438

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [3]:
### Update from CHATGPT

import json
import math

# Step 1: Load the GeoJSON data
with open('../kml/merged_test_1.geojson') as f:
    data = json.load(f)

# Step 2: Loop through the features
for feature in data['features']:
    if feature['geometry']['type'] == 'Polygon':
        # Step 3: Get the home plate coordinates
        home_plate = feature['geometry']['coordinates'][0][0]

        # Step 4: Loop through the vertices and calculate the distances
        for i in range(len(feature['geometry']['coordinates'][0])):
            vertex = feature['geometry']['coordinates'][0][i]
            x = (vertex[0] - home_plate[0]) * math.cos(home_plate[1])
            y = vertex[1] - home_plate[1]
            distance = math.sqrt(x**2 + y**2) * 3959 * 5280
            print(distance)

    elif feature['geometry']['type'] == 'MultiPolygon':
        # Step 3: Get the home plate coordinates
        home_plate = feature['geometry']['coordinates'][0][0][0]

        # Step 4: Loop through the vertices and calculate the distances
        for polygon in feature['geometry']['coordinates']:
            for i in range(len(polygon[0])):
                vertex = polygon[0][i]
                x = (vertex[0] - home_plate[0]) * math.cos(home_plate[1])
                y = vertex[1] - home_plate[1]
                distance = math.sqrt(x**2 + y**2) * 3959 * 5280
                print(distance)


0.0
9767.296182650602
9794.958248038753
9967.17886053238
4640.2454668859855
2664.5469101915405
2433.385096697594
1564.353998832172
1527.8358762505752
17768.661831723624
17483.11741407881
0.0
0.0
17483.11741407881
17409.45335956131
17300.40295435624
17272.082221454384
17330.290395182546
17266.627149716074
17106.6462007446
16906.982498089037
16753.051765130258
16551.264613797426
16379.768413375696
16031.475125049501
15638.264972717247
14832.813698017002
13874.099246009853
13097.650250157296
12582.649483872798
12075.185981971648
11559.348479851233
10686.866754452401
10009.29551680942
9781.30825866538
9767.296182650602
0.0
0.0
17867.721476174946
17735.446049961127
11532.259484436932
4207.698940254443
2041.3013858679142
2409.364581169501
3349.950778114291
9956.805544556903
14194.050148844384
14307.149034941098
0.0
0.0
14307.149034941098
16932.828216031416
17796.529364773374
18274.178170129584
19028.861532423347
19623.548230244007
20060.183467899104
20509.36349884554
20913.134403321033
21305

In [1]:
import json
from shapely.geometry import Polygon, mapping
from shapely.ops import unary_union

# read geojson file
with open('../kml/merged_test_1.geojson') as f:
    data = json.load(f)

# extract polygons and convert to shapely objects
polygons = []
for feature in data['features']:
    geometry = feature['geometry']
    if geometry['type'] == 'Polygon':
        polygons.append(Polygon(geometry['coordinates'][0]))
    elif geometry['type'] == 'MultiPolygon':
        for polygon in geometry['coordinates']:
            polygons.append(Polygon(polygon[0]))

# merge polygons
merged_polygon = unary_union(polygons)

# write merged polygon to geojson file
merged_feature = {'type': 'Feature', 'properties': {}, 'geometry': mapping(merged_polygon)}
merged_data = {'type': 'FeatureCollection', 'features': [merged_feature]}

with open('merged.geojson', 'w') as f:
    json.dump(merged_data, f)


In [1]:
import json
from shapely.geometry import Polygon, Point
from geojson import Point as GeojsonPoint

# Open the GeoJSON file
with open('../kml/merged_test_1.geojson', 'r') as f:
    data = json.load(f)

# Loop through the features
for feature in data['features']:
    if 'fop' in feature['properties'].keys() or 'Fop' in feature['properties'].keys() or 'FOP' in feature['properties'].keys():
        # Extract the coordinates of the first point in the feature
        coords = feature['geometry']['coordinates'][0]
        home_plate_coords = (coords[0][0], coords[0][1])
        # Create a Point object representing the home plate coordinates
        home_plate_point = Point(home_plate_coords)
        # Extract the fair territory coordinates and create a Polygon object
        fair_territory_coords = feature['geometry']['coordinates'][0]
        fair_territory_polygon = Polygon(fair_territory_coords)
        # Check if the home plate point is inside the fair territory polygon
        if fair_territory_polygon.contains(home_plate_point):
            # If the home plate is inside fair territory, print the coordinates
            print(f"The fair territory for this feature is: {fair_territory_coords}")
            print(f"The coordinates for home plate in this feature are: {home_plate_coords}")


In [3]:
import math

# Define home plate and outfield vertices
home_plate = (0, 0)
outfield_vertices = [(200, 0), (200, 200), (0, 200), (0, 0)]

# Define function to calculate distance between two points
def distance(p1, p2):
    return math.sqrt((p2[0]-p1[0])**2 + (p2[1]-p1[1])**2)

# Calculate distance from home plate to each outfield vertex
for i in range(len(outfield_vertices)):
    if i == 0:
        dist = distance(home_plate, outfield_vertices[i])
    elif i < len(outfield_vertices) - 1:
        dist += distance(outfield_vertices[i-1], outfield_vertices[i])
    else:
        dist += distance(outfield_vertices[i-1], outfield_vertices[i])
        dist += distance(home_plate, outfield_vertices[i])
    print(f"Degree {i*90} distance: {dist}")



Degree 0 distance: 200.0
Degree 90 distance: 400.0
Degree 180 distance: 600.0
Degree 270 distance: 800.0


In [4]:
#### create line segments from outfield vertices and save them in JSON format,

### Example from chatgpt

import json

# outfield vertices
outfield = [(0, 0), (0, 10), (5, 15), (10, 10), (10, 0)]

# generate line segments
line_segments = []
for i in range(len(outfield)):
    if i == len(outfield) - 1:
        line_segments.append({"start": outfield[i], "end": outfield[0]})
    else:
        line_segments.append({"start": outfield[i], "end": outfield[i+1]})

# save line segments as JSON
with open("line_segments.json", "w") as outfile:
    json.dump(line_segments, outfile)


In [5]:
import json

# Load the JSON file
with open('../kml/merged_test_1.geojson', 'r') as f:
    outfield_wall = json.load(f)

# Get the coordinates
coords = outfield_wall['features'][0]['geometry']['coordinates'][0]

# Define home plate as the first and last coordinate
home_plate = coords[0]
outfield_vertices = coords[1:-1]

print("Home plate:", home_plate)
print("Outfield vertices:", outfield_vertices)


Home plate: [-84.3395166, 41.9898236, 0.0]
Outfield vertices: [[-84.3406566, 41.9898485, 0.0], [-84.3406579, 41.9897867, 0.0], [-84.3406526, 41.9897179, 0.0], [-84.3399766, 41.989706, 0.0], [-84.3395529, 41.989697, 0.0], [-84.3394831, 41.989708, 0.0], [-84.3393651, 41.9897817, 0.0], [-84.3393423, 41.9898395, 0.0], [-84.3393222, 41.9906699, 0.0], [-84.3394898, 41.9906599, 0.0]]


In [9]:
import json

with open('../kml/merged_test_1.geojson', 'r') as f:
    data = json.load(f)

homeplate = data['features'][0]['geometry']['coordinates'][0]
outfield_vertices = []
for feature in data['features'][1:]:
    outfield_vertices.extend(feature['geometry']['coordinates'][:-1])

new_data = {
    "name": data["name"],
    "crs": data["crs"],
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [homeplate] + outfield_vertices + [homeplate]
            }
        }
    ]
}

with open('outfield.json', 'w') as f:
    json.dump(new_data, f)



In [10]:
import json
from math import sqrt

# Define a function to calculate distance in feet
def distance(point1, point2):
    return sqrt((point2[0]-point1[0])**2 + (point2[1]-point1[1])**2) * 364.5

# Load the JSON file
with open('../kml/merged_test_1.geojson', 'r') as f:
    outfield_wall = json.load(f)

# Get the coordinates
coords = outfield_wall['features'][0]['geometry']['coordinates'][0]

# Define home plate as the first and last coordinate
home_plate = coords[0]
outfield_vertices = coords[1:-1]

# Calculate the distance from home plate to each outfield vertex
for vertex in outfield_vertices:
    dist = distance(home_plate, vertex)
    print(f"Distance from home plate to {vertex}: {dist:.2f} feet")


Distance from home plate to [-84.3406566, 41.9898485, 0.0]: 0.42 feet
Distance from home plate to [-84.3406579, 41.9897867, 0.0]: 0.42 feet
Distance from home plate to [-84.3406526, 41.9897179, 0.0]: 0.42 feet
Distance from home plate to [-84.3399766, 41.989706, 0.0]: 0.17 feet
Distance from home plate to [-84.3395529, 41.989697, 0.0]: 0.05 feet
Distance from home plate to [-84.3394831, 41.989708, 0.0]: 0.04 feet
Distance from home plate to [-84.3393651, 41.9897817, 0.0]: 0.06 feet
Distance from home plate to [-84.3393423, 41.9898395, 0.0]: 0.06 feet
Distance from home plate to [-84.3393222, 41.9906699, 0.0]: 0.32 feet
Distance from home plate to [-84.3394898, 41.9906599, 0.0]: 0.30 feet


In [11]:
#### RESULTS LOOK OK, BEST SO FAR AT LEAST. The problem with the strange distances is probably because of some messed up coordinates in the geomentries

### IT could also be that it is looking at the foul territory 

import json
from math import radians, cos, sin, asin, sqrt

# Function to calculate the Haversine distance between two points on a sphere
def haversine(lat1, lon1, lat2, lon2):
    R = 20902231  # radius of Earth in feet
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    return R * c

# Load the JSON file
with open('../kml/merged_test_1.geojson', 'r') as f:
    outfield_wall = json.load(f)

# Get the coordinates
coords = outfield_wall['features'][0]['geometry']['coordinates'][0]

# Define home plate as the first and last coordinate
home_plate = coords[0]
outfield_vertices = coords[1:-1]

# Calculate the distance from home plate to each outfield vertex
for vertex in outfield_vertices:
    distance = haversine(home_plate[1], home_plate[0], vertex[1], vertex[0])
    print(f"Distance from home plate to {vertex}: {distance:.2f} feet")


Distance from home plate to [-84.3406566, 41.9898485, 0.0]: 309.25 feet
Distance from home plate to [-84.3406579, 41.9897867, 0.0]: 309.76 feet
Distance from home plate to [-84.3406526, 41.9897179, 0.0]: 310.43 feet
Distance from home plate to [-84.3399766, 41.989706, 0.0]: 131.90 feet
Distance from home plate to [-84.3395529, 41.989697, 0.0]: 47.22 feet
Distance from home plate to [-84.3394831, 41.989708, 0.0]: 43.14 feet
Distance from home plate to [-84.3393651, 41.9897817, 0.0]: 43.83 feet
Distance from home plate to [-84.3393423, 41.9898395, 0.0]: 47.62 feet
Distance from home plate to [-84.3393222, 41.9906699, 0.0]: 313.21 feet
Distance from home plate to [-84.3394898, 41.9906599, 0.0]: 305.18 feet


In [18]:
## Created byu chatgpt

### Try to integrate the block above into a loop togo through every firld in the json

import json
from math import sin, cos, sqrt, atan2, radians

# Load the JSON file
with open('filtered_baseball_fields.json', 'r') as f:
    ball_fields = json.load(f)

# Loop through every ball field
for field in ball_fields['features']:
    # Get the coordinates
    coords = field['geometry']['coordinates'][0]

    # Define home plate as the first and last coordinate
    home_plate = coords[0]
    outfield_vertices = coords[1:-1]

    # Loop through every outfield vertex
    for vertex in outfield_vertices:
        # Calculate the distance from home plate to the vertex in feet
        lat1 = radians(home_plate[1])
        lon1 = radians(home_plate[0])
        lat2 = radians(vertex[1])
        lon2 = radians(vertex[0])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = 20902231.97 * c  # 20902231.97 is the radius of the Earth in feet

        print(f"Distance from home plate to {vertex}: {distance:.2f} feet")


Distance from home plate to [-84.3394898, 41.9906599, 0.0]: 305.18 feet
Distance from home plate to [-84.3395904, 41.9906559, 0.0]: 304.29 feet
Distance from home plate to [-84.3397232, 41.9906469, 0.0]: 305.53 feet
Distance from home plate to [-84.3398278, 41.99064, 0.0]: 309.56 feet
Distance from home plate to [-84.3398828, 41.990639, 0.0]: 313.60 feet
Distance from home plate to [-84.3399538, 41.99063, 0.0]: 317.17 feet
Distance from home plate to [-84.3400518, 41.9906121, 0.0]: 322.19 feet
Distance from home plate to [-84.3401617, 41.9905881, 0.0]: 329.21 feet
Distance from home plate to [-84.340265, 41.9905642, 0.0]: 337.90 feet
Distance from home plate to [-84.3403441, 41.9905393, 0.0]: 344.26 feet
Distance from home plate to [-84.3404018, 41.9905184, 0.0]: 349.08 feet
Distance from home plate to [-84.340438, 41.9904914, 0.0]: 348.96 feet
Distance from home plate to [-84.3404554, 41.9904655, 0.0]: 345.88 feet
Distance from home plate to [-84.3404903, 41.9904107, 0.0]: 339.97 feet

In [19]:
### Saves the results to a csv file

import json
import csv
from math import sin, cos, sqrt, atan2, radians

# Load the JSON file
with open('filtered_baseball_fields.json', 'r') as f:
    ball_fields = json.load(f)

results = []
# Loop through every ball field
for field in ball_fields['features']:
    # Get the coordinates
    coords = field['geometry']['coordinates'][0]

    # Define home plate as the first and last coordinate
    home_plate = coords[0]
    outfield_vertices = coords[1:-1]

    # Loop through every outfield vertex
    for vertex in outfield_vertices:
        # Calculate the distance from home plate to the vertex in feet
        lat1 = radians(home_plate[1])
        lon1 = radians(home_plate[0])
        lat2 = radians(vertex[1])
        lon2 = radians(vertex[0])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = 20902231.97 * c  # 20902231.97 is the radius of the Earth in feet

        result = {"home_plate": home_plate, "vertex": vertex, "distance": distance}
        results.append(result)

# Save results to JSON file
with open('results.json', 'w') as f:
    json.dump(results, f)

# Save results to CSV file
with open('results.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Home Plate", "Vertex", "Distance (feet)"])
    for result in results:
        writer.writerow([result["home_plate"], result["vertex"], result["distance"]])


In [27]:
## In this modified code, I added a loop to calculate the distances for each outfield vertex, and then added the distances to a list. I also added the field name to the results. Finally, I changed the CSV output to include the field name and the distances in one row.


import json
import csv
from math import sin, cos, sqrt, atan2, radians

# Load the JSON file
with open('filtered_baseball_fields.json', 'r') as f:
    ball_fields = json.load(f)

results = []

# Loop through every ball field
for field in ball_fields['features']:
    # Get the name of the field
    field_name = field['properties']['Name']
    
    # Get the coordinates
    coords = field['geometry']['coordinates'][0]

    # Define home plate as the first and last coordinate
    home_plate = coords[0]
    outfield_vertices = coords[1:-1]

    distances = []
    # Loop through every outfield vertex
    for vertex in outfield_vertices:
        # Calculate the distance from home plate to the vertex in feet
        lat1 = radians(home_plate[1])
        lon1 = radians(home_plate[0])
        lat2 = radians(vertex[1])
        lon2 = radians(vertex[0])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = 20902231.97 * c  # 20902231.97 is the radius of the Earth in feet

        distances.append(distance)

    result = {"name": field_name, "distance": distances}
    results.append(result)

# Save results to JSON file
with open('results.json', 'w') as f:
    json.dump(results, f)

# Save results to CSV file
with open('results.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Field Name", "Distances (feet)"])
    for result in results:
        writer.writerow([result["name"]] + result["distance"])


In [16]:
### Looks like the loop is working but it is calulating for both the fair and foul polygons. I am going to try to drop all the foul teritory polygons before running the loop

import json

# Read in the JSON data
with open('../kml/merged_test_1.geojson', 'r') as f:
    data = json.load(f)

# Filter out the data for the "foul" geometry
features = [f for f in data['features'] if f['properties']['Name'].lower() == 'fop']
filtered_data = {
    'type': 'FeatureCollection',
    'name': 'filtered_test_1',
    'crs': data['crs'],
    'features': features
}

# Write the filtered data to a new JSON file
with open('filtered_baseball_fields.json', 'w') as f:
    json.dump(filtered_data, f)
